### 1.3.2 신경망 기본 아키텍쳐

머신러닝 라이브러리를 활용하지 않고, 파이썬 코드만으로 단순한 신경망을 구현해보겠다.

기본 신경망은 다음 요소로 구성된다.

- 입력 레이어 한 개(x)
- 은닉 레이어 다수
- 출력 레이어 한 개(y햇)
- 각 레이어 간 가중치(w)와 편향(b)
- 각 은닉 레이어의 활성화 함수(a)

### 1.3.3 파이썬만으로 신경망 만들기

In [1]:
import numpy as np

In [2]:
# 활성함수를 먼저 만들어주자.
def sigmoid(x):
    return 1.0/(1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)

class NeuralNetwork:
    
    # layer 2개를 가진 신경망에 대한 값 선언
    def __init__(self, x, y):
        self.input = x
        self.weight1 = np.random.rand(self.input.shape[1], 4)
        self.weight2 = np.random.rand(4, 1)
        self.y = y
        self.output = np.zeros(self.y.shape)
        
    # 1.3.3.1 훈련을 위한 순전파 과정
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weight1)) #인풋과 w1을 받아서 활성함수를 거쳐, layer1 결과물이 나온다.
        self.output = sigmoid(np.dot(self.layer1, self.weight2)) # layer1의 결과물과 w2를 받아서 활성함수를 거쳐, output이 나온다.
        
    # 1.3.3.2 평가를 위한 손실함수
    # SSE
    
    # 1.3.3.3 오차 개선을 위한 역전파 과정
    def backprop(self):
        # weight2와 weight1에 따른 손실 함수의 미분을 찾기 위해 연쇄 법칙을 활용
        d_weight2 = np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        d_weight1 = np.dot(self.input.T,(np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output),
                                                self.weight2.T) * sigmoid_derivative(self.layer1)))
        
        # 손실 함수의 미분 값을 사용해 가중치를 갱신한다.
        self.weight1 += d_weight1
        self.weight2 += d_weight2
        
if __name__ == "__main__":
    X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
    y = np.array([[0], [1], [1], [0]])
    nn = NeuralNetwork(X, y)
    
for i in range(1500):
    nn.feedforward()
    nn.backprop()

print(nn.output)

[[0.00949661]
 [0.96872454]
 [0.96845817]
 [0.03819881]]


### 1.5.2 케라스로 신경망 만들기

앞서 파이썬으로 간단한 신경망을 만들었다면, 이제 케라스로 똑같이 만들어보자.

케라스를 통한 신경망 구성 요소는 다음과 같다.

- 레이어 : 기본 빌딩 블록으로 레이어를 쌓아 모델을 생성
- 옵티마이저 : 훈련을 시키는 알고리즘(SGD, ADAM 등)
- 손실함수 : 오차 지표(MSE, CCE, BCE 등)

In [9]:
from keras.models import Sequential

In [10]:
# 레이어를 나란히 연결하기 위해 Sequential 을 먼저 선언해준다.
model = Sequential()

In [11]:
from keras.layers import Dense # 완전연결 레이어

In [12]:
# layer1
model.add(Dense(units=4, activation='sigmoid', input_dim=3)) # input_dim은 데이터셋의 변수가 몇 개인지 지정하는 것이다.
# 출력 레이어
model.add(Dense(units=1, activation='sigmoid'))

In [13]:
print(model.summary()) #param은 모델이 훈련해야 할 가중치와 편향 개수다.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 16        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
from keras import optimizers # 옵티마이저를 정의하자.

In [15]:
sgd = optimizers.SGD(lr=1) # SGD로 간다. lr은 learning rate.
model.compile(loss='mean_squared_error', optimizer=sgd) # 손실함수는 MSE

In [16]:
# 이전과 동일한 데이터셋을 만들어서 돌려보자.
np.random.seed(9)
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0], [1], [1], [0]])

In [17]:
model.fit(X, y, epochs=1500, verbose=False)
print(model.predict(X))

[[0.07159124]
 [0.94394034]
 [0.9117078 ]
 [0.08230858]]


전혀 수학적 코드 없이도 케라스 만으로 신경망을 돌릴 수 있다.